In [154]:
import pandas as pd

In [155]:
raw_data = pd.DataFrame()
for i in range(1, 6):
    df_temp = pd.read_csv(f"../data/raw/wrds_bond_returns{i}.csv.gz", compression="gzip")
    raw_data = pd.concat([raw_data, df_temp], ignore_index=True)

/var/folders/hw/dfkspcjj1bv53cx3_42lwrtc0000gn/T/ipykernel_25353/1798892931.py:3: DtypeWarning: Columns (55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp = pd.read_csv(f"../data/raw/wrds_bond_returns{i}.csv.gz", compression="gzip")
/var/folders/hw/dfkspcjj1bv53cx3_42lwrtc0000gn/T/ipykernel_25353/1798892931.py:3: DtypeWarning: Columns (55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp = pd.read_csv(f"../data/raw/wrds_bond_returns{i}.csv.gz", compression="gzip")
/var/folders/hw/dfkspcjj1bv53cx3_42lwrtc0000gn/T/ipykernel_25353/1798892931.py:3: DtypeWarning: Columns (15,55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp = pd.read_csv(f"../data/raw/wrds_bond_returns{i}.csv.gz", compression="gzip")


In [156]:
df = raw_data.copy()
# Lowercase column names
cols = [str_.lower() for str_ in df.columns]
df.columns = cols

In [157]:
print(cols)

['date', 'issue_id', 'cusip', 'bond_sym_id', 'bsym', 'isin', 'company_symbol', 'bond_type', 'security_level', 'conv', 'offering_date', 'offering_amt', 'offering_price', 'principal_amt', 'maturity', 'treasury_maturity', 'coupon', 'day_count_basis', 'dated_date', 'first_interest_date', 'last_interest_date', 'ncoups', 'amount_outstanding', 'r_sp', 'r_mr', 'r_fr', 'n_sp', 'n_mr', 'n_fr', 'rating_num', 'rating_cat', 'rating_class', 't_date', 't_volume', 't_dvolume', 't_spread', 't_yld_pt', 'yield', 'price_eom', 'price_ldm', 'price_l5m', 'gap', 'coupmonth', 'nextcoup', 'coupamt', 'coupacc', 'multicoups', 'ret_eom', 'ret_ldm', 'ret_l5m', 'tmt', 'remcoups', 'duration', 'defaulted', 'default_date', 'default_type', 'reinstated', 'reinstated_date']


In [158]:

# Filter bond types
print("initial shape:", df.shape)
# Delete defaulted bonds
df = df[df['defaulted']=='N']
print("after removing defaulted bonds:", df.shape)

# Keep only senior bonds
df = df[df['security_level'] == 'SEN']
print("after keeping only senior bonds:", df.shape)



initial shape: (3854028, 58)
after removing defaulted bonds: (3840829, 58)
after keeping only senior bonds: (3567979, 58)


In [159]:
# Select relevant columns
select_cols = ['date', 
               'cusip', # CUSIP identifier
               'company_symbol', # Company ticker symbol
               'tmt', # Time to Maturity
               'coupon', # Coupon Rate
               't_spread', # Average bid-ask spread
               'yield', # Yield to Maturity
               'ret_eom', # Total Return End of Month (Including Coupons)
               'rating_cat', # Weighted Average Rating Category
]
df = df[select_cols].copy()

In [160]:
# Check if there is duplicate keys
if df.duplicated(subset=['date', 'cusip']).sum() > 0:
    print("Warning: There are duplicate keys in the data!")
else:
    print("No duplicate found, use (date, cusip) as unique identifier.")

No duplicate found, use (date, cusip) as unique identifier.


In [161]:
# transform data types
df.loc[:, ['tmt', 'coupon']] = df.loc[:, ['tmt', 'coupon']].astype(float) 
df.loc[:, 'coupon'] = df.loc[:, 'coupon'] / 100.0  # convert coupon to decimal

for col in ['t_spread', 'yield', 'ret_eom']:
    df[col] = df[col].str.rstrip('%').astype(float) / 100.0

df.loc[:, ['t_spread', 'yield', 'ret_eom']] = df.loc[:, ['t_spread', 'yield', 'ret_eom']].apply(pd.to_numeric, errors='coerce')

In [162]:
# Deal with anomalies

# extreme yields
df.loc[df['yield'] > 1, 'yield'] = float('nan')  # yields greater than 200% as high yield bonds should have different logic
df.loc[df['yield'] < -1, 'yield'] = float('nan')  # yields less than -100% are likely data errors

# fill nan ratings with 'NR' (not rated)
df['rating_cat'] = df['rating_cat'].fillna('NR')


In [163]:
# one-hot coding for rating_cat
from sklearn.preprocessing import OneHotEncoder

oh = OneHotEncoder()
rating_cat_oh = oh.fit_transform(df[['rating_cat']]).toarray()
rating_cat_oh_df = pd.DataFrame(rating_cat_oh, columns=[f"rating_{cat}" for cat in oh.categories_[0]]).astype(int)
df = pd.concat([df.reset_index(drop=True), rating_cat_oh_df.reset_index(drop=True)], axis=1)

# Delete NR category as it does not provide useful information
df = df.drop(columns=['rating_NR'])

In [164]:
rating_dict = {'AAA': 1,
               'AA': 2,
               'A': 3,
               'BBB': 4,
               'BB': 5,
               'B': 6,
               'CCC': 7,
               'CC': 8,
               'C': 9,
               'D': 10,
               'NR': float('nan')}

In [165]:
# Find rating downgrades and upgrades
df = df.sort_values(by=['cusip', 'date'])
df.loc[:, 'upgrade'] = df.groupby('cusip')['rating_cat'].transform(lambda x: x.map(rating_dict).diff() < 0)
df.loc[:, 'downgrade'] = df.groupby('cusip')['rating_cat'].transform(lambda x: x.map(rating_dict).diff() > 0)

# Fill NaN values in upgrade and downgrade columns with 0
df.loc[:, ['upgrade', 'downgrade']] = df.loc[:, ['upgrade', 'downgrade']].fillna(0).astype(int)

# Delete rating_cat column
df = df.drop(columns=['rating_cat'])


/var/folders/hw/dfkspcjj1bv53cx3_42lwrtc0000gn/T/ipykernel_25353/1010259344.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 0 0 ... 0 0 0]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, ['upgrade', 'downgrade']] = df.loc[:, ['upgrade', 'downgrade']].fillna(0).astype(int)
/var/folders/hw/dfkspcjj1bv53cx3_42lwrtc0000gn/T/ipykernel_25353/1010259344.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 0 0 ... 0 0 0]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, ['upgrade', 'downgrade']] = df.loc[:, ['upgrade', 'downgrade']].fillna(0).astype(int)


In [166]:
# Date column to datetime and use the end of month
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d') + pd.offsets.MonthEnd(0) 

In [167]:
# Read Interest Rate Data
ir_raw_data = pd.read_csv("../data/raw/interest_rate.csv.gz", compression="gzip")


In [168]:
# Select Treasury Constant Maturity 3-Month Rate as short-term interest rate and 10-Year Rate as long-term interest rate
ir_data = ir_raw_data[['date', 'gs3m', 'gs10']].copy()
ir_data.loc[:, ['gs3m', 'gs10']] = ir_data.loc[:, ['gs3m', 'gs10']] / 100.0  # convert to decimal
ir_data.loc[:, 'term_spread'] = ir_data['gs10'] - ir_data['gs3m']



In [169]:
# Date column to datetime and use the end of month
ir_data['date'] = pd.to_datetime(ir_data['date'], format='%Y-%m-%d') + pd.offsets.MonthEnd(0)
 

In [170]:
# merge interest rate data with bond data
df = pd.merge(df, ir_data[['date', 'gs3m', 'term_spread']], on='date', how='left')

In [ ]:
# Shift predictor variables by 1 month to avoid look-ahead bias
constant_cols = ['date', 'cusip', 'company_symbol', 'tmt', 'coupon', 'ret_eom']
lagged_cols = df.columns.difference(constant_cols)
df.sort_values(by=['cusip', 'date'], inplace=True)
df[lagged_cols] = df.groupby('cusip')[lagged_cols].shift(1)

In [176]:
df.dropna(axis=0, inplace=True)

In [172]:
df.to_parquet("../data/processed/bond_data_processed.parquet", index=False)

In [173]:
summary = df.describe()
summary.to_csv("../data/processed/bond_data_summary.csv")

In [174]:
# NaN check
nan_summary = df.isna().sum()
print("NaN summary:")
print(nan_summary / len(df))  # percentage of NaN values

NaN summary:
date              0.000000
cusip             0.000000
company_symbol    0.000189
tmt               0.002783
coupon            0.000886
t_spread          0.499684
yield             0.050220
ret_eom           0.040485
rating_A          0.040469
rating_AA         0.040469
rating_AAA        0.040469
rating_B          0.040469
rating_BB         0.040469
rating_BBB        0.040469
rating_C          0.040469
rating_CC         0.040469
rating_CCC        0.040469
rating_D          0.040469
upgrade           0.040469
downgrade         0.040469
gs3m              0.040469
term_spread       0.040469
dtype: float64


In [177]:
print(len(df))
print(df['cusip'].nunique())
print(df['company_symbol'].nunique())

1760852
74191
3187
